### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

## Example 1. Get all patient encounters from 2015.

In [1]:
from sqlalchemy import select

from cyclops.query import mimic
from cyclops.query import utils as query_utils
from cyclops.query.interface import QueryInterface

patients = mimic.patients(years=[2015])
patients.run()
print(f"{len(patients.data)} rows extracted!")

2022-05-03 22:58:50,074 INFO cyclops.config  - Log file is /home/amritk/projects/cyclops/log.log
2022-05-03 22:58:50,421 INFO cyclops.query.utils - Log file is /home/amritk/projects/cyclops/log.log
2022-05-03 22:58:50,423 INFO cyclops.utils.profile - Log file is /home/amritk/projects/cyclops/log.log
2022-05-03 22:58:50,424 INFO cyclops.orm     - Log file is /home/amritk/projects/cyclops/log.log
2022-05-03 22:58:50,426 INFO cyclops.query.interface - Log file is /home/amritk/projects/cyclops/log.log
2022-05-03 22:58:51,089 INFO cyclops.orm     - Database setup, ready to run queries!
2022-05-03 22:58:52,298 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:58:52,299 INFO cyclops.utils.profile - Finished executing function run_query in 1.198511 s


98084 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [2]:
patients = mimic.patients(years=[2015])
patients_schizophrenia = mimic.diagnoses(substring="schizophrenia", patients=patients)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")
patients_schizophrenia.data["icd_title"].value_counts()

2022-05-03 22:58:52,947 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:58:52,949 INFO cyclops.utils.profile - Finished executing function run_query in 0.599514 s


5239 rows extracted!


Schizophrenia, unspecified                                                 3116
Unspecified schizophrenia, unspecified                                      954
Paranoid schizophrenia                                                      451
Unspecified schizophrenia, chronic                                          153
Paranoid type schizophrenia, unspecified                                    123
Catatonic schizophrenia                                                     103
Catatonic type schizophrenia, unspecified                                    65
Paranoid type schizophrenia, chronic with acute exacerbation                 52
Other schizophrenia                                                          43
Paranoid type schizophrenia, chronic                                         35
Other specified types of schizophrenia, chronic with acute exacerbation      26
Other specified types of schizophrenia, unspecified                          25
Catatonic type schizophrenia, chronic   

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [3]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(diagnoses.query.c.icd_title.regexp_match("(?=.*schizophrenia)(?=.*chronic)"))
    .subquery()
)
patients_schizophrenia_chronic = diagnoses
patients_schizophrenia_chronic.query = subquery
patients_schizophrenia_chronic.run()
print(f"{len(patients_schizophrenia_chronic.data)} rows extracted!")

2022-05-03 22:58:54,053 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:58:54,054 INFO cyclops.utils.profile - Finished executing function run_query in 1.086604 s


310 rows extracted!


## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [4]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(query_utils.starts_with(diagnoses.query.c.icd_title, "Paranoid"))
    .subquery()
)
patients_schizophrenia = diagnoses
patients_schizophrenia.query = subquery
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-05-03 22:58:58,121 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:58:58,122 INFO cyclops.utils.profile - Finished executing function run_query in 4.050461 s


710 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [5]:
patients = mimic.patients()
patients_f209 = mimic.diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

2022-05-03 22:59:00,024 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:59:00,025 INFO cyclops.utils.profile - Finished executing function run_query in 1.886616 s


14818 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [6]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2022-05-03 22:59:01,488 INFO cyclops.orm     - Query returned successfully!
2022-05-03 22:59:01,489 INFO cyclops.utils.profile - Finished executing function run_query in 1.442194 s


52482 rows extracted!


## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [7]:
patients = mimic.patients(years=[2015])
patients_vitals = mimic.events(category="Routine Vital Signs", patients=patients)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

2022-05-03 23:00:08,432 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:00:08,434 INFO cyclops.utils.profile - Finished executing function run_query in 66.923356 s


100 rows extracted!


## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [8]:
patients = mimic.patients(years=[2009])
patients_hemo_labs = mimic.events(
    category="labs", patients=patients, names=["hemoglobin"]
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

2022-05-03 23:00:35,938 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:00:35,940 INFO cyclops.utils.profile - Finished executing function run_query in 27.496512 s


100 rows extracted!


## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [9]:
patients = mimic.patients(years=[2009])
patients_sodium_labs = mimic.events(
    category="labs", patients=patients, substring="sodium"
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

2022-05-03 23:01:03,258 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:01:03,259 INFO cyclops.utils.profile - Finished executing function run_query in 27.310337 s


100 rows extracted!


## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [10]:
patients = mimic.patients(years=[2009])
patients_respiratory = mimic.events(category="Respiratory", patients=patients)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

2022-05-03 23:02:09,529 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:02:09,530 INFO cyclops.utils.profile - Finished executing function run_query in 66.253171 s


100 rows extracted!


## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 1000 rows.

In [11]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium_heart_rate = mimic.events(
    names="Heart Rate", patients=patients_delirium
)
patients_delirium_heart_rate.run(limit=1000)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

2022-05-03 23:02:39,124 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:02:39,125 INFO cyclops.utils.profile - Finished executing function run_query in 29.560723 s


1000 rows extracted!


## Example 12. Get d_items dimension table, for quick reference.

In [13]:
event_lookup_table = mimic.get_lookup_table("event_labels")
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")

2022-05-03 23:03:41,827 INFO cyclops.orm     - Query returned successfully!
2022-05-03 23:03:41,828 INFO cyclops.utils.profile - Finished executing function run_query in 0.029290 s


3861 rows extracted!
